In [ ]:
# function that takes 1 to 4 inputs and creates a different balanced datasets
# DATA0.0 360 degree images from Google dataset
# DATA0.1 360 degree images of our own

# DATA0 360 degree images with fire extinguishers (this is what was in the validation folder)
# DATA1 normal images from internet (oldouz labels )
# DATA2 deformed images with data augmentation (ari labels)

# import

In [1]:
import os
# importing shutil module  
import shutil
import random

## google drive mount

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# directories

In [3]:
#change work directory
os.chdir("/content/drive/My Drive/DATASET/")

In [4]:
# change work directory and check on where you are 
!ls

DATA_0	DATA_1	DATA_2	datasets_overview.xlsx	data.zip  test


In [7]:
# var my directory
mydir = os.getcwd()
mydir

'/content/drive/My Drive/DATASET'

## data

In [8]:
# store subfolders
DATASET = os.listdir(mydir)
DATA = sorted(DATASET)[:3]
DATA

['DATA_0', 'DATA_1', 'DATA_2']

In [12]:
DATASET = os.listdir(mydir)
DATASET


['DATA_2', 'DATA_0', 'DATA_1', 'test', 'data.zip', 'datasets_overview.xlsx']

## deformation

In [13]:
DEFORMATION = ["High","Low", "None"]
DEF = 1
DEFORMATION[DEF:DEF+1]

['Low']

## check file type

.png not admitted

In [14]:
#taking the extension out

def find_all_extension_infolder(destination):
  SplitTypes=[]
  for file in os.listdir(destination):
      SplitTypes.append(file.split('.')[-1])
  set_extensions = list(set(SplitTypes))
  return (set_extensions)

In [15]:
# checking extensions out 
DATA_SIZE = []
for sub in DATA:
  set_extensions = find_all_extension_infolder(sub)
  print (set_extensions)

['jpg', 'xml']
['jpg', 'xml']
['jpg', 'xml']


In [ ]:
#delete_extensions = ['jpeg', 'png']

## check folders

take as images input only ,jpg files
controls that for each images there is one annotation and vice versa
discard the rest

In [16]:
# count the number of elements in each subfolder
# rememeber images and annotations are together so the number should be even 
def get_images_annotation_in_folders(DATA):
  """
  DATA is a vector of sub folders
  DATA_SIZE, data_dict,wrong_files = get_images_annotation_in_folders(DATA)
  """
  DATA_SIZE = []
  data_dict =  dict()
  wrong_files = list()
  for sub in DATA:
    sub_dir =  (mydir + '/' +sub)
    all_files = os.listdir(sub_dir)
    DATA_SIZE.append(len(all_files))
    annotations = [file for file in all_files if file.endswith(".xml")]
    #images = [file for file in all_files if (file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".png")) ] 
    images = [file for file in all_files if (file.endswith(".jpg"))]
    others = []
    #others = [file for file in others if not file in annotations ]
    others = [file for file in all_files if (file.endswith(".jpeg") or file.endswith(".png"))]
    wrong_files.extend(others)
    data_dict[sub] = (images, annotations)
    check = ""
    if len(annotations) != len(images):
      check = "<------------ shite"
    print ("%s contains %s files: %s images and %s annotations ,else %s %s" %(sub_dir , len(all_files), len(images), len(annotations),len(others),check))
    #if len(others) > 0:
      #print (others)
  return DATA_SIZE, data_dict, wrong_files

In [17]:
DATA_SIZE, data_dict,wrong_files = get_images_annotation_in_folders(DATA)

/content/drive/My Drive/DATASET/DATA_0 contains 208 files: 104 images and 104 annotations ,else 0 
/content/drive/My Drive/DATASET/DATA_1 contains 800 files: 400 images and 400 annotations ,else 0 
/content/drive/My Drive/DATASET/DATA_2 contains 800 files: 400 images and 400 annotations ,else 0 


In [18]:
DATA_SIZE

[208, 800, 800]

In [19]:
# remove unmatching images / annotations
# refresh the data_dict!
images_to_delete = []
annotations_to_delete = []
for k in data_dict.keys():
  print ("---%s----" %k)
  images, annotations = data_dict[k] # refresh the data_dict!
  print (len(images), len(annotations))
  set_a = set([ x[:-4] for x in images])
  set_b = set([ x[:-4] for x in annotations])
  images_to_delete = set_a-set_b
  annotations_to_delete = set_b-set_a
  print (len(images_to_delete),len(annotations_to_delete))

  for im in images_to_delete:
      fullpath = mydir+'/'+ k +'/'+ im + ".jpg"
      print (fullpath)
      os.remove(fullpath)

  for annotation in annotations_to_delete:
      fullpath = mydir+'/'+ k +'/'+ annotation + ".xml"
      print (fullpath)
      os.remove(fullpath)

---DATA_0----
104 104
0 0
---DATA_1----
400 400
0 0
---DATA_2----
400 400
0 0


## check after cleaning status

In [20]:
# check on the cleaning 
DATA_SIZE, data_dict, wrong_files = get_images_annotation_in_folders(DATA)

/content/drive/My Drive/DATASET/DATA_0 contains 208 files: 104 images and 104 annotations ,else 0 
/content/drive/My Drive/DATASET/DATA_1 contains 800 files: 400 images and 400 annotations ,else 0 
/content/drive/My Drive/DATASET/DATA_2 contains 800 files: 400 images and 400 annotations ,else 0 


In [21]:
# test!
images, annotations = data_dict["DATA_2"][0],data_dict["DATA_2"][1]
images, annotations = sorted(images), sorted(annotations)
print (images[2], annotations[2])

2020-10-05T130842.160-low-zoom25.jpg 2020-10-05T130842.160-low-zoom25.xml


we do NOT take the .jpeg because:
- error in training of the model
- the datasets are nicely balanced this way!

# create folder "test" in mydir.
all the models will be tested on the same images, 9 from each dataset
test are handpicked to be rapresentative of the results
only once, not needed anylonger

In [22]:
def collect_from_directory(dir):
  """
  images, annotations = collect_from_directory(dir)
  prints number of jpg images in a directory and number of xml files in a directory
  returns images and annotations
  """
  tot_files = os.listdir(dir)
  annotations = [file for file in tot_files if file.endswith(".xml")]
  images = [file for file in tot_files if file.endswith(".jpg")]
  print ("in %s there are annotations: %s, images: %s" %(dir, len(images), len(annotations)))
  return images, annotations

In [23]:
test_folder = mydir + "/test/"
try: 
    # create directory if it doesnt exist
    os.mkdir(test_folder) 
    print ("-created sub_dir: %s" %test_folder)
    test_subdir = []
    for d in DEFORMATION[DEF:DEF+1]:
      test_sub = test_folder+d
      test_subdir.append(test_sub)
      # create sub directory if it doesnt exist
      try: 
        os.mkdir(test_sub) 
        print ("created sub_dir: %s" %test_sub)

      except OSError as error: 
          print(error)  
except OSError as error: 
  print(error)
  # check number of files in folders
  for dir in os.listdir('/content/drive/My Drive/DATASET/test/'):
    images, annotations = collect_from_directory(test_folder +dir)
    




[Errno 17] File exists: '/content/drive/My Drive/DATASET/test/'
in /content/drive/My Drive/DATASET/test/Low there are annotations: 9, images: 9
in /content/drive/My Drive/DATASET/test/None there are annotations: 9, images: 9
in /content/drive/My Drive/DATASET/test/High there are annotations: 9, images: 9


# MAIN

In [24]:
# check on the cleaning 
DATA_SIZE, data_dict, wrong_files = get_images_annotation_in_folders(DATA)

/content/drive/My Drive/DATASET/DATA_0 contains 208 files: 104 images and 104 annotations ,else 0 
/content/drive/My Drive/DATASET/DATA_1 contains 800 files: 400 images and 400 annotations ,else 0 
/content/drive/My Drive/DATASET/DATA_2 contains 800 files: 400 images and 400 annotations ,else 0 


## sampling function

In [25]:
def give_sampled_files(k,sampling,mydir):
  """
  move_files, move_files_check,remaining_files = give_sampled_files(k,sampling)
  k is the dictionary key (eg DATA0)
  sampling are the indexes choosen to be sampled
  """
  # to train 
  curr_dir = mydir + '/' +k
  all_files = (os.listdir(curr_dir))

  # check for unmatch images-annotations
  
  #DATA_SIZE_k, data_dict_k,wrong_files_k = get_images_annotation_in_folders(str(data_dict[k]))

  # clean up remaining files
  


  # to train 
  all_images, all_annotations = data_dict[k][0],data_dict[k][1]
  all_images, all_annotations = sorted(all_images), sorted(all_annotations) #sorting is important!
  move_files = []
  move_files_check = []
  for i in sampling:
    this_im = all_images[i]
    this_ann = all_annotations[i]
    if this_im[:-4] == this_ann[:-4]: 
      move_files.append( curr_dir +'/' + this_im)
      move_files.append ( curr_dir +'/' + this_ann)
      move_files_check.append(this_im)
      move_files_check.append(this_ann)
    else:
      print ("error in the order of files!")
      
  remaining_files = [ (curr_dir +'/' + file) for file in all_files if not file in move_files_check]
  
  print ("sampled %s imgs for train" %(len(move_files)/2))
  print ("remaining %s imgs are for validation " %(len(remaining_files)/2))
  return move_files,move_files_check,remaining_files

## dataset parameters




In [27]:
# set % of images in dataset to go for training
P = []
#P = [90,90,0]# percentage of images

# number of images from each dataset
if len(P)>0:
  N = [p*DATA_SIZE[i]/200 for i,p in enumerate(P)]
#or set directly the number of files
N = [90,370,0]

if (len(N) or len(P)) != len (DATA):
  print (" probabilities and number must be as many as the datasets considered")

## dataset name

In [28]:
# name the dataset
degree360,notdeformed, deformed = int(N[0]),int(N[1]),int(N[2])
sum_N = int(sum (N))
dataset_name = "tot_%s-panorama_%s-notdeformed_%s-deformed_%s" %(sum_N,
                                                                     degree360,
                                                                     notdeformed, 
                                                                     deformed
                                                                     )
print (dataset_name)

tot_460-panorama_90-notdeformed_370-deformed_0


In [29]:
mydir

'/content/drive/My Drive/DATASET'

## create subfolders in dataset name

In [30]:
# create new dataset folder
main_new = mydir +"/"+dataset_name
try: 
  os.mkdir(main_new)
  print ("created dir: %s" %main_new)
except OSError as error: 
    print(error)  
    
# define subfolders
subfolders_name = ["/train/","/validation/","/myresults/"]
subsubfolders_name = ["annotations/", "images/"]
subfolders = []
for sub in subfolders_name:
  print ("")
  print (sub)
  path = main_new + sub
  subfolders.append(path)
  try: 
    os.mkdir(path) 
    print ("-created sub_dir: %s" %path)

  except OSError as error: 
      print(error) 

  if sub in subfolders_name[:2]:
    for subsub in subsubfolders_name:
      path2 = path + subsub 
      try: 
        os.mkdir(path2) 
        print ("--created sub_dir: %s" %path2)
      except OSError as error: 
        print(error) 

train_folder,validation_folder =  subfolders[0],subfolders[1]

print ("")
print ("check")
print (train_folder + subsubfolders_name[0])
print (validation_folder)
print (dataset_name)

created dir: /content/drive/My Drive/DATASET/tot_460-panorama_90-notdeformed_370-deformed_0

/train/
-created sub_dir: /content/drive/My Drive/DATASET/tot_460-panorama_90-notdeformed_370-deformed_0/train/
--created sub_dir: /content/drive/My Drive/DATASET/tot_460-panorama_90-notdeformed_370-deformed_0/train/annotations/
--created sub_dir: /content/drive/My Drive/DATASET/tot_460-panorama_90-notdeformed_370-deformed_0/train/images/

/validation/
-created sub_dir: /content/drive/My Drive/DATASET/tot_460-panorama_90-notdeformed_370-deformed_0/validation/
--created sub_dir: /content/drive/My Drive/DATASET/tot_460-panorama_90-notdeformed_370-deformed_0/validation/annotations/
--created sub_dir: /content/drive/My Drive/DATASET/tot_460-panorama_90-notdeformed_370-deformed_0/validation/images/

/myresults/
-created sub_dir: /content/drive/My Drive/DATASET/tot_460-panorama_90-notdeformed_370-deformed_0/myresults/

check
/content/drive/My Drive/DATASET/tot_460-panorama_90-notdeformed_370-deformed

### correcting directory files
they dont belong to test!

In [31]:
check_statement = False
remove_statement = False

In [32]:
if check_statement == True:
  correcting_folder = "/content/drive/My Drive/DATASET/test/None/"
  all_added_files = (os.listdir(correcting_folder))
  content = os.listdir(DATA[0])
  content.extend(os.listdir(DATA[1]))
  content.extend(os.listdir(DATA[2]))
  all_right_files =  [file for file in all_added_files if not file in content]
  all_right_files
  print ("there are %s all right files" ,len(all_right_files))
  fuck_files =  [file for file in all_added_files if file in content]
  print ("there are %s wrong files" ,len(fuck_files))
  if remove_statement == True:
    for file in fuck_files:
      fullpath = correcting_folder+file
      os.remove(fullpath)



## define images and annotations folders path for train and validation

In [33]:
im_train_folder = train_folder + subsubfolders_name[1]
ann_train_folder = train_folder + subsubfolders_name[0]
im_val_folder = validation_folder + subsubfolders_name[1]
ann_val_folder = validation_folder + subsubfolders_name[0]

print (im_train_folder)
print (ann_train_folder)
print (im_val_folder)
print (ann_val_folder)

/content/drive/My Drive/DATASET/tot_460-panorama_90-notdeformed_370-deformed_0/train/images/
/content/drive/My Drive/DATASET/tot_460-panorama_90-notdeformed_370-deformed_0/train/annotations/
/content/drive/My Drive/DATASET/tot_460-panorama_90-notdeformed_370-deformed_0/validation/images/
/content/drive/My Drive/DATASET/tot_460-panorama_90-notdeformed_370-deformed_0/validation/annotations/


## sample files for training, the rest is for validation

In [35]:
files_in_validation_already = 0
for i in range(len(DATA)):
  k = DATA[i]
  
  tot_size = DATA_SIZE[i]/2
  train_N = int(N[i])

  print ("")
  print (" %s there are in total %s " %(k, tot_size))
 
  list_indexes = list(range(train_N))
  sampling = random.sample(list_indexes, k=train_N)

  move_files, move_files_check,remaining_files = give_sampled_files(k,sampling,mydir)

  if tot_size != len(move_files)/2 +len(remaining_files)/2:
    print ("error! lost some files? ")
  else:


    #sampled files go to train set
    print ("copying files in train")
    for file in move_files:
      extension = file.split('.')[-1]
      if extension == "jpg":
        shutil.copy2(file, im_train_folder) # target filename is /dst/dir/file.ext
      elif extension == "xml":
        shutil.copy2(file,ann_train_folder)
    
    #remaining file go to validation set
    print ("copying files in eval")

    # count how many files are already in validation
    #files_in_validation_already = os.listdir(im_val_folder)

    # take only as many as to reach 200 for validation
    max_validation = 200
    difference = max_validation- (files_in_validation_already)
    print ("there are already %s files for evaluation" %((files_in_validation_already)))

    limit = min(int(len(remaining_files)/2),difference)

    #print ("")
    print ("chosing from %s remaining files" %len(remaining_files/2))
    #for i in remaining_files:
      #print (i)
    count_ann = 0
    count_im = 0
    #print ("")


    for file in (remaining_files)[:limit]:
      extension = file.split('.')[-1]
      name = file[:-4]

      file_im = name +".jpg"
      file_ann = name + ".xml"
      #if extension == "jpg":
      shutil.copy2(file_im, im_val_folder) # target filename is /dst/dir/file.ext
      count_im += 1
      #elif extension == "xml":
      shutil.copy2(file_ann,ann_val_folder)
      count_ann += 1
    files_in_validation_already += limit
    print ("added further %s img and %s ann for validation" %(count_im,count_ann))

print ("")
print (mydir +'/'+ dataset_name)




 DATA_0 there are in total 104.0 
sampled 90.0 imgs for train
remaining 14.0 imgs are for validation 
copying files in train
copying files in eval
there are already 0 files for evaluation
chosing from 28 remaining files
added further 14 img and 14 ann for validation

 DATA_1 there are in total 400.0 
sampled 370.0 imgs for train
remaining 30.0 imgs are for validation 
copying files in train
copying files in eval
there are already 14 files for evaluation
chosing from 60 remaining files
added further 30 img and 30 ann for validation

 DATA_2 there are in total 400.0 
sampled 0.0 imgs for train
remaining 400.0 imgs are for validation 
copying files in train
copying files in eval
there are already 44 files for evaluation
chosing from 800 remaining files
added further 156 img and 156 ann for validation

/content/drive/My Drive/DATASET/tot_460-panorama_90-notdeformed_370-deformed_0


In [49]:
mydir

'/content/drive/My Drive/DATASET'

# zip files

if in the shared drive, move them manually to my own!

In [55]:
#!ls "/content/drive/My Drive/data/" # in my own drive
!ls "/content/drive/My Drive/DATASET/" # in shared drive
zipping_directory = "fire extinguisher"
#storing_folder = "/content/drive/My Drive/data/fire extinguisher/"
storing_folder = "/content/drive/My Drive/DATASET/fire extinguisher/"

DATA_0	datasets_overview.xlsx	tot_460-panorama_90-notdeformed_370-deformed_0
DATA_1	data.zip
DATA_2	test


In [ ]:
import shutil
zipping_directory = "test"
#storing_folder = mydir +'/'+ dataset_name
storing_folder = mydir +'/'+ zipping_directory

In [ ]:
shutil.make_archive(
  zipping_directory, 
  'zip',           # the archive format - or tar, bztar, gztar 
  root_dir=storing_folder,   # root for archive - current working dir if None
  base_dir=storing_folder)   # start archiving from here - cwd if None too

OSError: ignored

# New Section

In [ ]:
# define subfolders
#subfolders = ["/test/", "/train/"]

In [ ]:
# check naming 
for i in range(len(subfolders)):
  print (mydir + subfolders[i])

/content/drive/My Drive/DATASET/test/
/content/drive/My Drive/DATASET/train/


# 